# Modul 3 Sains Data: Metode Imputasi, Encoding Data Kategorik, Regresi Linier

Kembali ke [Sains Data](./saindat2024genap.qmd)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Install `scikit-learn` dengan:

`!pip install scikit-learn`

Lalu import `sklearn`:

In [7]:
import sklearn

## Import Dataset

Untuk praktikum kali ini, kita akan menggunakan dataset "California Housing Prices" (housing.csv) yang bisa didownload dari salah satu sumber berikut:

* [Direct link (langsung dari GitHub Pages ini)](./housing.csv)

* Kaggle: <https://www.kaggle.com/datasets/camnugent/california-housing-prices>

Kemudian, baca sebagai dataframe:

In [3]:
df = pd.read_csv("./housing.csv")

Mari kita lihat isinya:

In [4]:
df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


Apakah ada *missing value*?

In [5]:
df.isna().sum()

longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64

In [6]:
df[df["total_bedrooms"].isna()]

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
290,-122.16,37.77,47.0,1256.0,NaN,570.0,218.0,4.3750,161900.0,NEAR BAY
341,-122.17,37.75,38.0,992.0,NaN,732.0,259.0,1.6196,85100.0,NEAR BAY
538,-122.28,37.78,29.0,5154.0,NaN,3741.0,1273.0,2.5762,173400.0,NEAR BAY
563,-122.24,37.75,45.0,891.0,NaN,384.0,146.0,4.9489,247100.0,NEAR BAY
696,-122.10,37.69,41.0,746.0,NaN,387.0,161.0,3.9063,178400.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20267,-119.19,34.20,18.0,3620.0,NaN,3171.0,779.0,3.3409,220500.0,NEAR OCEAN
20268,-119.18,34.19,19.0,2393.0,NaN,1938.0,762.0,1.6953,167400.0,NEAR OCEAN
20372,-118.88,34.17,15.0,4260.0,NaN,1701.0,669.0,5.1033,410700.0,<1H OCEAN
20460,-118.75,34.29,17.0,5512.0,NaN,2734.0,814.0,6.6073,258100.0,<1H OCEAN


Perhatikan bahwa tipe datanya adalah `int64` atau bilangan bulat.

Dari 20640 baris, ada satu kolom/fitur (`total_bedrooms`) dengan 207 missing value.

Secara umum, ada dua cara untuk menangani *missing value*:

1. Menghapus baris-baris yang memiliki *missing value*, dengan `df.dropna()`

2. Melakukan metode imputasi

Karena banyaknya *missing value* relatif sedikit, sebenarnya tidak masalah apabila baris-baris tersebut cukup dihapus saja. Namun, kita akan mempelajari metode imputasi.

## Metode Imputasi

### Median

In [7]:
df_fill_median = df.copy()

In [8]:
df["total_bedrooms"].median()

435.0

In [9]:
bedrooms_median = df["total_bedrooms"].median()
print(bedrooms_median)

435.0


In [10]:
df_fill_median["total_bedrooms"] = df_fill_median["total_bedrooms"].fillna(bedrooms_median)

In [11]:
df_fill_median.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

Cara lain, menggunakan scikit-learn:

In [12]:
df_fill_median2 = df.copy()

In [13]:
from sklearn.impute import SimpleImputer

In [14]:
median_imputer = SimpleImputer(
    missing_values=np.nan, strategy='median'
)

In [15]:
df_fill_median2[["total_bedrooms"]] = median_imputer.fit_transform(
    df_fill_median2[["total_bedrooms"]]
)

In [16]:
df_fill_median2.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

### Modus

In [17]:
df_fill_mode = df.copy()

In [18]:
df["total_bedrooms"].mode()

0    280.0
Name: total_bedrooms, dtype: float64

In [20]:
df["total_bedrooms"].mode()[0]

280.0

In [21]:
bedrooms_mode = df["total_bedrooms"].mode()[0]
print(bedrooms_mode)

280.0


In [22]:
df_fill_mode["total_bedrooms"] = df_fill_mode["total_bedrooms"].fillna(bedrooms_mode)

In [23]:
df_fill_mode.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

Cara lain, menggunakan scikit-learn:

In [24]:
df_fill_mode2 = df.copy()

In [25]:
mode_imputer = SimpleImputer(
    missing_values=np.nan, strategy='most_frequent'
)

In [28]:
df_fill_mode2[["total_bedrooms"]] = mode_imputer.fit_transform(
    df_fill_mode2[["total_bedrooms"]]
)

In [29]:
df_fill_mode2.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

### Mean (rata-rata)

In [30]:
df_fill_mean = df.copy()

In [31]:
df_fill_mean["total_bedrooms"].mean()

537.8705525375618

In [32]:
np.round(df_fill_mean["total_bedrooms"].mean())

538.0

In [33]:
bedrooms_mean = np.round(df_fill_mean["total_bedrooms"].mean())
print(bedrooms_mean)

538.0


In [34]:
df_fill_mean["total_bedrooms"] = df_fill_mean["total_bedrooms"].fillna(bedrooms_mean)

In [35]:
df_fill_mean.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

Cara lain, menggunakan scikit-learn:

In [36]:
df_fill_mean2 = df.copy()

In [37]:
mean_imputer = SimpleImputer(
    missing_values=np.nan, strategy='mean'
)

In [38]:
df_fill_mean2[["total_bedrooms"]] = mean_imputer.fit_transform(
    df_fill_mean2[["total_bedrooms"]]
)

In [39]:
df_fill_mean2.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

### KNNImputer

In [40]:
from sklearn.impute import KNNImputer

In [41]:
knn_imputer = KNNImputer(n_neighbors=3)

In [42]:
df_fill_knn = df.copy()

KNN Imputer memerlukan kolom-kolom lainnya sebagai acuan, dan hanya bisa bekerja dengan data numerik. Sehingga, kita perlu mem-filter terlebih dahulu kolom-kolom numerik dari dataset kita.

In [43]:
df_fill_knn.select_dtypes(include='number')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


In [44]:
df_fill_knn.select_dtypes(include='number').columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value'],
      dtype='object')

In [45]:
num_col = df_fill_knn.select_dtypes(include='number').columns

In [46]:
df_fill_knn[num_col] = knn_imputer.fit_transform(df_fill_knn[num_col])

In [47]:
df_fill_knn.isna().sum()

longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

## Perbandingan Metode Imputasi

## Encoding Data Kategorik